<a href="https://colab.research.google.com/github/The-cheater/Deep_Learning_Models/blob/main/MMCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

# Unzip GAF and MTF images
gaf_zip = "/content/GAF_Images_224x224.zip"
mtf_zip = "/content/MTF_Images_224x224.zip"

with zipfile.ZipFile(gaf_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/GAF_Images_224x224")

with zipfile.ZipFile(mtf_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/MTF_Images_224x224")


In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
import os, random

class GAFMTFGenerator(Sequence):
    def __init__(self, gaf_dir, mtf_dir, batch_size=32, image_size=(224, 224), shuffle=True):
        self.gaf_dir = gaf_dir
        self.mtf_dir = mtf_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle

        self.class_names = sorted(os.listdir(gaf_dir))
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(self.class_names)

        self.samples = []
        for label in self.class_names:
            gaf_path = os.path.join(gaf_dir, label)
            mtf_path = os.path.join(mtf_dir, label)
            for fname in os.listdir(gaf_path):
                if fname.endswith(".png") and os.path.exists(os.path.join(mtf_path, label, fname)):
                    self.samples.append((os.path.join(gaf_path, label, fname),
                                         os.path.join(mtf_path, label, fname),
                                         label))
        print(f"📦 Total image pairs loaded: {len(self.samples)}")
        if self.shuffle:
            random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples) // self.batch_size

    def __getitem__(self, idx):
        batch = self.samples[idx * self.batch_size : (idx + 1) * self.batch_size]
        gaf_batch, mtf_batch, labels = [], [], []

        for gaf_path, mtf_path, label in batch:
            gaf_img = img_to_array(load_img(gaf_path, target_size=self.image_size)) / 255.0
            mtf_img = img_to_array(load_img(mtf_path, target_size=self.image_size)) / 255.0
            gaf_batch.append(gaf_img)
            mtf_batch.append(mtf_img)
            labels.append(label)

        gaf_batch = np.array(gaf_batch)
        mtf_batch = np.array(mtf_batch)
        labels = self.label_encoder.transform(labels)
        labels = tf.keras.utils.to_categorical(labels, num_classes=len(self.class_names))

        return {"gaf_input": gaf_batch, "mtf_input": mtf_batch}, labels

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.samples)


In [22]:
model = build_mmcnn(input_shape=(224, 224, 3), num_classes=4)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [24]:
gaf_path = "/content/GAF_Images_224x224/GAF_Images_224x224/Left Stride Interval"
mtf_path = "/content/MTF_Images_224x224/MTF_Images_224x224/Left Stride Interval"

train_gen = GAFMTFGenerator(gaf_path, mtf_path, batch_size=16)
model.fit(train_gen, epochs=20)


📦 Total image pairs loaded: 0


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0